In [ ]:
# Preparation, test pytesseract
from PIL import Image
import pytesseract #pip install pytesseract first

# Load an image using Pillow (PIL)
image = Image.open(r'C:\Users\FeiFei\Pictures\12.jpg')

# Perform OCR on the image
text = pytesseract.image_to_string(image)

print(text)
     

In [1]:
# python -m pip install feedparser requests trafilatura pytesseract playwright pillow
# python -m playwright install

# Step 1 — Install required packages
# !pip install requests beautifulsoup4 trafilatura pytesseract pillow tqdm

In [4]:
# Step 2 — Set up and test Tesseract path (Windows only)
import pytesseract

# 👇 Adjust this path if your Tesseract is elsewhere
pytesseract.pytesseract.tesseract_cmd = r"C:\Users\FeiFei\AppData\Local\Programs\Tesseract-OCR\tesseract.exe"

# Test it
print("✅ Tesseract version:", pytesseract.get_tesseract_version())


✅ Tesseract version: 5.5.0.20241111


In [6]:
# Step 3 — Import libraries and define scraper functions

import requests
from bs4 import BeautifulSoup
import trafilatura
from tqdm import tqdm
import pytesseract
from PIL import Image
import json
from io import BytesIO
import os

BASE_URL = "https://arxiv.org"
CATEGORY = "cs.CL"   # 👈 Change this to any subcategory (e.g. cs.LG, math.ST)
N_PAPERS = 50         # You can increase later (e.g. 200)


In [7]:
# Step 4 — Helper functions

def fetch_arxiv_links(category=CATEGORY, n=N_PAPERS):
    """Fetch latest paper abstract URLs from arXiv list page."""
    print(f"🔎 Fetching latest {n} papers from {category}...")
    url = f"{BASE_URL}/list/{category}/pastweek?show={n}"
    res = requests.get(url)
    soup = BeautifulSoup(res.text, "html.parser")

    links = []
    for dt in soup.find_all("dt"):
        link = dt.find("a", title="Abstract")
        if link:
            links.append(BASE_URL + link["href"])
    return links


def parse_arxiv_page(url):
    """Parse a single arXiv abstract page."""
    res = requests.get(url)
    soup = BeautifulSoup(res.text, "html.parser")

    # Extract metadata
    title = soup.find("h1", class_="title").get_text(strip=True).replace("Title:", "")
    authors = soup.find("div", class_="authors").get_text(strip=True).replace("Authors:", "")
    date = soup.find("div", class_="dateline").get_text(strip=True)

    # Try to extract abstract text with Trafilatura
    downloaded = trafilatura.fetch_url(url)
    abstract_text = ""
    if downloaded:
        abstract_text = trafilatura.extract(downloaded, include_comments=False) or ""

    # If Trafilatura fails or gives empty text, fallback to OCR
    if not abstract_text.strip():
        img = soup.find("img")
        if img and "src" in img.attrs:
            img_url = BASE_URL + img["src"]
            img_res = requests.get(img_url)
            image = Image.open(BytesIO(img_res.content))
            abstract_text = pytesseract.image_to_string(image)

    return {
        "url": url,
        "title": title,
        "authors": authors,
        "date": date,
        "abstract": abstract_text.strip()
    }


In [8]:
# Step 5 — Run scraper and save output

links = fetch_arxiv_links()
papers = []

for link in tqdm(links, desc="Scraping papers"):
    try:
        paper = parse_arxiv_page(link)
        papers.append(paper)
    except Exception as e:
        print(f"⚠️ Error parsing {link}: {e}")

# Save to JSON
output_file = "arxiv_clean.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(papers, f, ensure_ascii=False, indent=2)

size_kb = os.path.getsize(output_file) / 1024
print(f"\n📁 Saved {len(papers)} papers to {output_file} ({size_kb:.1f} KB)")


🔎 Fetching latest 50 papers from cs.CL...


Scraping papers: 100%|█████████████████████████████████████████████████████████████████| 50/50 [00:41<00:00,  1.20it/s]


📁 Saved 50 papers to arxiv_clean.json (149.1 KB)


In [2]:
# Task 2
"""Goal

You’ll take the PDFs of the same papers scraped in Task 1,
convert each page to images, extract text via Tesseract OCR,
and save the results as .txt files under a pdf_ocr/ folder."""

# Step 1 — Install dependencies

# !pip install pytesseract pdf2image pillow tqdm

'Goal\n\nYou’ll take the PDFs of the same papers scraped in Task 1,\nconvert each page to images, extract text via Tesseract OCR,\nand save the results as .txt files under a pdf_ocr/ folder.'

In [13]:
import os

input_folder = "pdfs"       # folder for your PDF files
output_folder = "pdf_ocr"   # output text folder

os.makedirs(input_folder, exist_ok=True)
os.makedirs(output_folder, exist_ok=True)

print("✅ Folders ready:")
print(" -", os.path.abspath(input_folder))
print(" -", os.path.abspath(output_folder))


✅ Folders ready:
 - C:\Users\FeiFei\Documents\AI Engineer\Homework\Sophie_HW 2\pdfs
 - C:\Users\FeiFei\Documents\AI Engineer\Homework\Sophie_HW 2\pdf_ocr


In [20]:
# Auto-download PDFs from your Task 1 JSON
import json, requests, os

input_folder = "pdfs"
os.makedirs(input_folder, exist_ok=True)

# Load Task 1 output
with open("arxiv_clean.json", "r", encoding="utf-8") as f:
    papers = json.load(f)

for i, paper in enumerate(papers[:5], 1):  # limit to first 5 PDFs for testing
    pdf_url = paper["url"].replace("/abs/", "/pdf/") + ".pdf"
    pdf_path = os.path.join(input_folder, f"paper_{i}.pdf")
    print(f"⬇️ Downloading {pdf_url}")
    try:
        r = requests.get(pdf_url)
        with open(pdf_path, "wb") as f:
            f.write(r.content)
    except Exception as e:
        print(f"⚠️ Failed: {pdf_url} -> {e}")

print("✅ PDFs downloaded to:", os.path.abspath(input_folder))


⬇️ Downloading https://arxiv.org/pdf/2510.26802v1.pdf
⬇️ Downloading https://arxiv.org/pdf/2510.26790v1.pdf
⬇️ Downloading https://arxiv.org/pdf/2510.26788v1.pdf
⬇️ Downloading https://arxiv.org/pdf/2510.26787v1.pdf
⬇️ Downloading https://arxiv.org/pdf/2510.26768v1.pdf
✅ PDFs downloaded to: C:\Users\FeiFei\Documents\AI Engineer\Homework\Sophie_HW 2\pdfs


In [16]:
# Step 2 — Configure paths
import pytesseract
from pdf2image import convert_from_path
import os
from tqdm import tqdm

# 👇 Update if necessary
pytesseract.pytesseract.tesseract_cmd = r"C:\Users\FeiFei\AppData\Local\Programs\Tesseract-OCR\tesseract.exe"
poppler_path = r"C:\Users\FeiFei\Documents\AI Engineer\Release-25.07.0-0\poppler-25.07.0\Library\bin"  # <-- change to your Poppler bin folder

input_folder = "pdfs"       # Folder where your arXiv PDFs are stored
output_folder = "pdf_ocr"   # Output TXT folder
os.makedirs(output_folder, exist_ok=True)


In [22]:
import os

input_folder = "pdfs"
pdf_files = [f for f in os.listdir(input_folder) if f.lower().endswith(".pdf")]

print(f"📄 Found {len(pdf_files)} PDF(s) in {os.path.abspath(input_folder)}:")
for f in pdf_files:
    print(" -", f)


📄 Found 5 PDF(s) in C:\Users\FeiFei\Documents\AI Engineer\Homework\Sophie_HW 2\pdfs:
 - paper_1.pdf
 - paper_2.pdf
 - paper_3.pdf
 - paper_4.pdf
 - paper_5.pdf


In [21]:
# Step 3 — OCR function
from PIL import Image

def ocr_pdf_to_text(pdf_path, output_txt):
    """Convert a single PDF to text using Tesseract OCR."""
    text_output = []

    # Convert PDF to a list of images (one per page)
    pages = convert_from_path(pdf_path, dpi=300, poppler_path=poppler_path)

    for i, page in enumerate(pages, start=1):
        text = pytesseract.image_to_string(page)
        text_output.append(f"\n\n==== Page {i} ====\n{text}")

    # Combine pages and write to file
    with open(output_txt, "w", encoding="utf-8") as f:
        f.write("\n".join(text_output))


In [23]:
# Step 4 — Batch OCR for all PDFs
pdf_files = [f for f in os.listdir(input_folder) if f.lower().endswith(".pdf")]

for pdf in tqdm(pdf_files, desc="🔍 OCR Processing"):
    input_path = os.path.join(input_folder, pdf)
    output_txt = os.path.join(output_folder, os.path.splitext(pdf)[0] + ".txt")

    try:
        ocr_pdf_to_text(input_path, output_txt)
    except Exception as e:
        print(f"⚠️ Error on {pdf}: {e}")


🔍 OCR Processing: 100%|█████████████████████████████████████████████████████████████████| 5/5 [07:49<00:00, 93.88s/it]


In [ ]:
# Step 5 — (Optional) Preserve layout
text = pytesseract.image_to_string(
    page,
    config="--oem 1 --psm 1 preserve_interword_spaces=1"
)

In [ ]:
""" Task 3:Automatic Speech Recognition (ASR) with YouTube audio 
+ OCR for any embedded text in video slides, saving a .jsonl with timestamps.

Task 3 Overview

Download short NLP conference talk videos from YouTube (10 talks, ~3 min each).

Extract audio using yt-dlp.

If slides or video frames have text, extract using Tesseract OCR.

Save everything to .jsonl with timestamps.

Tools:

yt-dlp — download audio/video.

pytesseract — OCR any slides.

whisper (OpenAI) — ASR transcription.
"""

In [1]:
# Step 0 — Install dependencies
# !pip install yt-dlp pytesseract Pillow tqdm openai-whisper


In [2]:
# Step 1 — Configure paths and imports
import os
from pathlib import Path
import yt_dlp
from PIL import Image
import pytesseract
import whisper
import json
from tqdm import tqdm

# --------------------------
# Folders
video_folder = "talks_videos"
audio_folder = "talks_audio"
os.makedirs(video_folder, exist_ok=True)
os.makedirs(audio_folder, exist_ok=True)

# Tesseract path (Windows)
pytesseract.pytesseract.tesseract_cmd = r"C:\Users\FeiFei\AppData\Local\Programs\Tesseract-OCR\tesseract.exe"


In [3]:
# Step 2 — Download YouTube videos / audio
yt_urls = [
    "https://www.youtube.com/watch?v=-moW9jvvMr4",
    "https://www.youtube.com/watch?v=arj7oStGLkU",
    # add up to 10 talk URLs
]

ydl_opts = {
    'format': 'bestaudio/best',
    'outtmpl': os.path.join(audio_folder, '%(title)s.%(ext)s'),
    'noplaylist': True,
    'quiet': False
}

for url in yt_urls:
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([url])


[youtube] Extracting URL: https://www.youtube.com/watch?v=-moW9jvvMr4
[youtube] -moW9jvvMr4: Downloading webpage
[youtube] -moW9jvvMr4: Downloading android sdkless player API JSON
[youtube] -moW9jvvMr4: Downloading tv client config
[youtube] -moW9jvvMr4: Downloading tv player API JSON
[youtube] -moW9jvvMr4: Downloading web safari player API JSON
[youtube] -moW9jvvMr4: Downloading player c6d7bdc9-main


         player = https://www.youtube.com/s/player/c6d7bdc9/player_ias.vflset/en_US/base.js
         n = vyamxBSnjb-zZCoT ; player = https://www.youtube.com/s/player/c6d7bdc9/player_ias.vflset/en_US/base.js
         Please report this issue on  https://github.com/yt-dlp/yt-dlp/issues?q= , filling out the appropriate issue template. Confirm you are on the latest version using  yt-dlp -U


[youtube] -moW9jvvMr4: Downloading m3u8 information
[info] -moW9jvvMr4: Downloading 1 format(s): 251
[download] Sleeping 4.00 seconds as required by the site...
[download] Destination: talks_audio\A Simple Way to Break a Bad Habit ｜ Judson Brewer ｜ TED.webm
[download] 100% of    7.35MiB in 00:00:02 at 2.48MiB/s   
[youtube] Extracting URL: https://www.youtube.com/watch?v=arj7oStGLkU
[youtube] arj7oStGLkU: Downloading webpage
[youtube] arj7oStGLkU: Downloading android sdkless player API JSON
[youtube] arj7oStGLkU: Downloading tv client config
[youtube] arj7oStGLkU: Downloading tv player API JSON
[youtube] arj7oStGLkU: Downloading web safari player API JSON
[youtube] arj7oStGLkU: Downloading player c6d7bdc9-main


         player = https://www.youtube.com/s/player/c6d7bdc9/player_ias.vflset/en_US/base.js
         n = lt43S0HxD2pCs2Fr ; player = https://www.youtube.com/s/player/c6d7bdc9/player_ias.vflset/en_US/base.js
         Please report this issue on  https://github.com/yt-dlp/yt-dlp/issues?q= , filling out the appropriate issue template. Confirm you are on the latest version using  yt-dlp -U


[youtube] arj7oStGLkU: Downloading m3u8 information
[info] arj7oStGLkU: Downloading 1 format(s): 251
[download] Sleeping 4.00 seconds as required by the site...
[download] Destination: talks_audio\Inside the Mind of a Master Procrastinator ｜ Tim Urban ｜ TED.webm
[download] 100% of   12.42MiB in 00:00:07 at 1.72MiB/s   


In [4]:
# Optional: check files
for f in os.listdir(audio_folder):
    if f.lower().endswith((".mp3", ".m4a", ".wav", ".webm")):
        path = os.path.join(audio_folder, f)
        print(path, os.path.exists(path))

talks_audio\A Simple Way to Break a Bad Habit ｜ Judson Brewer ｜ TED.webm True
talks_audio\Inside the Mind of a Master Procrastinator ｜ Tim Urban ｜ TED.webm True
talks_audio\talk1.webm True
talks_audio\talk2.webm True


In [11]:
# Optional: change file name

import shutil

for i, f in enumerate(os.listdir(audio_folder), start=1):
    ext = os.path.splitext(f)[1]
    old_path = os.path.join(audio_folder, f)
    new_path = os.path.join(audio_folder, f"talk{i}{ext}")
    shutil.move(old_path, new_path)
    print(f"{old_path} → {new_path}")

talks_audio\A Simple Way to Break a Bad Habit ｜ Judson Brewer ｜ TED.webm → talks_audio\talk1.webm
talks_audio\Inside the Mind of a Master Procrastinator ｜ Tim Urban ｜ TED.webm → talks_audio\talk2.webm


In [5]:
# Optional: convert to .wav
for f in os.listdir(audio_folder):
    if f.lower().endswith((".m4a", ".webm")):
        src = os.path.join(audio_folder, f)
        dst = os.path.splitext(src)[0] + ".wav"
        !ffmpeg -y -i "{src}" "{dst}"
        print(f"Converted {src} → {dst}")


Converted talks_audio\A Simple Way to Break a Bad Habit ｜ Judson Brewer ｜ TED.webm → talks_audio\A Simple Way to Break a Bad Habit ｜ Judson Brewer ｜ TED.wav


ffmpeg version N-121584-ge59d964a3c-20251101 Copyright (c) 2000-2025 the FFmpeg developers
  built with gcc 15.2.0 (crosstool-NG 1.28.0.1_403899e)
  configuration: --prefix=/ffbuild/prefix --pkg-config-flags=--static --pkg-config=pkg-config --cross-prefix=x86_64-w64-mingw32- --arch=x86_64 --target-os=mingw32 --enable-gpl --enable-version3 --disable-debug --enable-shared --disable-static --disable-w32threads --enable-pthreads --enable-iconv --enable-zlib --enable-libxml2 --enable-libvmaf --enable-fontconfig --enable-libharfbuzz --enable-libfreetype --enable-libfribidi --enable-vulkan --enable-libshaderc --enable-libvorbis --disable-libxcb --disable-xlib --disable-libpulse --enable-opencl --enable-gmp --enable-lzma --enable-amf --enable-libaom --enable-libaribb24 --enable-avisynth --enable-chromaprint --enable-libdav1d --enable-libdavs2 --enable-libdvdread --enable-libdvdnav --disable-libfdk-aac --enable-ffnvcodec --enable-cuda-llvm --enable-frei0r --enable-libgme --enable-libkvazaar --e

Converted talks_audio\Inside the Mind of a Master Procrastinator ｜ Tim Urban ｜ TED.webm → talks_audio\Inside the Mind of a Master Procrastinator ｜ Tim Urban ｜ TED.wav


ffmpeg version N-121584-ge59d964a3c-20251101 Copyright (c) 2000-2025 the FFmpeg developers
  built with gcc 15.2.0 (crosstool-NG 1.28.0.1_403899e)
  configuration: --prefix=/ffbuild/prefix --pkg-config-flags=--static --pkg-config=pkg-config --cross-prefix=x86_64-w64-mingw32- --arch=x86_64 --target-os=mingw32 --enable-gpl --enable-version3 --disable-debug --enable-shared --disable-static --disable-w32threads --enable-pthreads --enable-iconv --enable-zlib --enable-libxml2 --enable-libvmaf --enable-fontconfig --enable-libharfbuzz --enable-libfreetype --enable-libfribidi --enable-vulkan --enable-libshaderc --enable-libvorbis --disable-libxcb --disable-xlib --disable-libpulse --enable-opencl --enable-gmp --enable-lzma --enable-amf --enable-libaom --enable-libaribb24 --enable-avisynth --enable-chromaprint --enable-libdav1d --enable-libdavs2 --enable-libdvdread --enable-libdvdnav --disable-libfdk-aac --enable-ffnvcodec --enable-cuda-llvm --enable-frei0r --enable-libgme --enable-libkvazaar --e

Converted talks_audio\talk1.webm → talks_audio\talk1.wav


ffmpeg version N-121584-ge59d964a3c-20251101 Copyright (c) 2000-2025 the FFmpeg developers
  built with gcc 15.2.0 (crosstool-NG 1.28.0.1_403899e)
  configuration: --prefix=/ffbuild/prefix --pkg-config-flags=--static --pkg-config=pkg-config --cross-prefix=x86_64-w64-mingw32- --arch=x86_64 --target-os=mingw32 --enable-gpl --enable-version3 --disable-debug --enable-shared --disable-static --disable-w32threads --enable-pthreads --enable-iconv --enable-zlib --enable-libxml2 --enable-libvmaf --enable-fontconfig --enable-libharfbuzz --enable-libfreetype --enable-libfribidi --enable-vulkan --enable-libshaderc --enable-libvorbis --disable-libxcb --disable-xlib --disable-libpulse --enable-opencl --enable-gmp --enable-lzma --enable-amf --enable-libaom --enable-libaribb24 --enable-avisynth --enable-chromaprint --enable-libdav1d --enable-libdavs2 --enable-libdvdread --enable-libdvdnav --disable-libfdk-aac --enable-ffnvcodec --enable-cuda-llvm --enable-frei0r --enable-libgme --enable-libkvazaar --e

Converted talks_audio\talk2.webm → talks_audio\talk2.wav


ffmpeg version N-121584-ge59d964a3c-20251101 Copyright (c) 2000-2025 the FFmpeg developers
  built with gcc 15.2.0 (crosstool-NG 1.28.0.1_403899e)
  configuration: --prefix=/ffbuild/prefix --pkg-config-flags=--static --pkg-config=pkg-config --cross-prefix=x86_64-w64-mingw32- --arch=x86_64 --target-os=mingw32 --enable-gpl --enable-version3 --disable-debug --enable-shared --disable-static --disable-w32threads --enable-pthreads --enable-iconv --enable-zlib --enable-libxml2 --enable-libvmaf --enable-fontconfig --enable-libharfbuzz --enable-libfreetype --enable-libfribidi --enable-vulkan --enable-libshaderc --enable-libvorbis --disable-libxcb --disable-xlib --disable-libpulse --enable-opencl --enable-gmp --enable-lzma --enable-amf --enable-libaom --enable-libaribb24 --enable-avisynth --enable-chromaprint --enable-libdav1d --enable-libdavs2 --enable-libdvdread --enable-libdvdnav --disable-libfdk-aac --enable-ffnvcodec --enable-cuda-llvm --enable-frei0r --enable-libgme --enable-libkvazaar --e

In [3]:
# !where ffmpeg
# !ffmpeg -version

In [4]:
# import whisper

# model = whisper.load_model("base")

# audio_files = [os.path.join(audio_folder, f) for f in os.listdir(audio_folder)
#                if f.lower().endswith(( ".webm"))]
#                # ((".mp3", ".m4a", ".wav", ".webm"))]

# for audio_path in audio_files:
#     print("Transcribing:", audio_path)
#     result = model.transcribe(audio_path)
#     print(result["text"])


In [9]:
# Step 3 — ASR transcription with Whisper
model = whisper.load_model("base")  # you can use "small", "medium", or "large"

audio_files = list(Path(audio_folder).glob("*.*"))
transcripts = []

for audio_path in tqdm(audio_files, desc="Transcribing audio"):
    result = model.transcribe(str(audio_path))
    
    # Collect segments with timestamps
    for segment in result['segments']:
        transcripts.append({
            "file": audio_path.name,
            "start": segment['start'],
            "end": segment['end'],
            "text": segment['text']
        })


Transcribing audio: 100%|███████████████████████████████████████████████████████████████| 2/2 [03:53<00:00, 116.63s/it]


In [5]:
# !pip install opencv-python

In [11]:
# Step 4 — Optional: OCR from video frames / slides
import cv2

def extract_frames(video_path, every_n_seconds=5):
    """Yield frames from video every n seconds."""
    cap = cv2.VideoCapture(str(video_path))
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_count = 0
    frames = []
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        if frame_count % int(fps*every_n_seconds) == 0:
            frames.append(frame)
        frame_count += 1
    cap.release()
    return frames

# Example OCR extraction
import numpy as np

ocr_texts = []
for video_file in Path(video_folder).glob("*.mp4"):
    frames = extract_frames(video_file, every_n_seconds=5)
    for frame in frames:
        img = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        text = pytesseract.image_to_string(img)
        if text.strip():
            ocr_texts.append({
                "video_file": video_file.name,
                "ocr_text": text.strip()
            })


In [12]:
# Step 5 — Save transcripts + OCR to JSONL
output_file = "talks_transcripts.jsonl"

with open(output_file, "w", encoding="utf-8") as f:
    # save ASR segments
    for seg in transcripts:
        f.write(json.dumps({"type": "asr", **seg}, ensure_ascii=False) + "\n")
    
    # save OCR segments
    for ocr in ocr_texts:
        f.write(json.dumps({"type": "ocr", **ocr}, ensure_ascii=False) + "\n")

print(f"✅ Saved transcripts to {output_file}")


✅ Saved transcripts to talks_transcripts.jsonl


In [ ]:
"""
Task 4 is essentially a post-processing / data cleaning pipeline that consolidates everything you did in Tasks 1‑3. 
Let’s break it down carefully and outline a lightweight, robust Jupyter-friendly implementation.
"""

In [ ]:
# Step 0 — Prerequisites
# pip install langdetect datasketch tqdm regex

In [23]:
# Step 1 — Merge all task outputs
import json
import os

corpus = []

# Task 1
with open("arxiv_clean.json", "r", encoding="utf-8") as f:
    corpus.extend(json.load(f))  # expects list of {url, title, abstract, authors, date}

# Task 2
pdf_folder = "pdf_ocr"
for fname in os.listdir(pdf_folder):
    if fname.endswith(".txt"):
        with open(os.path.join(pdf_folder, fname), "r", encoding="utf-8") as f:
            text = f.read()
        corpus.append({"source": fname, "text": text})

# Task 3
with open("talks_transcripts.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        corpus.append({"source": "talk", **json.loads(line)})


In [24]:
# Step 2 — Language Detection
from langdetect import detect, DetectorFactory
DetectorFactory.seed = 0  # deterministic

for doc in corpus:
    try:
        doc["lang"] = detect(doc.get("text", doc.get("abstract", "")))
    except:
        doc["lang"] = "unknown"


In [25]:
# Step 3 — Remove HTML / noise
import re

def clean_html(text):
    # remove HTML tags
    text = re.sub(r"<[^>]+>", " ", text)
    # remove multiple spaces/newlines
    text = re.sub(r"\s+", " ", text)
    return text.strip()

for doc in corpus:
    field = "text" if "text" in doc else "abstract"
    doc[field] = clean_html(doc[field])


In [26]:
# Step 4 — Deduplication with MinHash
from datasketch import MinHash, MinHashLSH

threshold = 0.7  # similarity threshold
lsh = MinHashLSH(threshold=threshold, num_perm=128)

minhashes = []
unique_docs = []

for i, doc in enumerate(corpus):
    field = "text" if "text" in doc else "abstract"
    words = set(doc[field].split())
    m = MinHash(num_perm=128)
    for w in words:
        m.update(w.encode("utf-8"))
    minhashes.append(m)
    # query LSH for duplicates
    if i == 0:
        unique_docs.append(doc)
        lsh.insert(str(i), m)
    else:
        if not lsh.query(m):
            unique_docs.append(doc)
            lsh.insert(str(i), m)

print(f"Deduplicated: {len(unique_docs)} / {len(corpus)}")
corpus = unique_docs


Deduplicated: 468 / 479


In [27]:
# Step 5 — Remove PII
pii_patterns = [
    r"[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+",  # emails
    r"\b(?:\d[ -]*?){13,16}\b",                          # credit cards
    r"\b(?:\+?\d{1,3}[ -]?)?(?:\(?\d{2,3}\)?[ -]?)?\d{3,4}[ -]?\d{3,4}\b"  # phone numbers
]

pii_removed_count = 0

def remove_pii(text):
    global pii_removed_count
    for pat in pii_patterns:
        matches = re.findall(pat, text)
        pii_removed_count += len(matches)
        text = re.sub(pat, "[REDACTED]", text)
    return text
    
# def remove_pii(text):
#     for pat in pii_patterns:
#         text = re.sub(pat, "[REDACTED]", text)
#     return text

for doc in corpus:
    field = "text" if "text" in doc else "abstract"
    doc[field] = remove_pii(doc[field])



In [28]:
print(pii_removed_count)

89


In [30]:
# Step 6 — Remove repetitive n‑grams
# For example, remove trigrams repeated consecutively:
repetitive_ngrams_removed = 0
def remove_repetitive_ngrams(text, n=3):
    global repetitive_ngrams_removed
    tokens = text.split()
    cleaned = []
    i = 0
    while i < len(tokens):
        ngram = tokens[i:i+n]
        # check next n tokens
        if i+n*2 <= len(tokens) and tokens[i+n:i+n*2] == ngram:
            repetitive_ngrams_removed += 1
            i += n  # skip repetition
        else:
            cleaned.append(tokens[i])
            i += 1
    return " ".join(cleaned)

for doc in corpus:
    field = "text" if "text" in doc else "abstract"
    doc[field] = remove_repetitive_ngrams(doc[field])

print(f"Repetitive n-grams removed: {repetitive_ngrams_removed}")

Repetitive n-grams removed: 7


In [20]:
# Step 7 — Save cleaned corpus & stats
# Save
def get_text(doc):
    if "text" in doc:
        return doc["text"]
    elif "abstract" in doc:
        return doc["abstract"]
    else:
        return ""  # fallback empty

# Save cleaned corpus
with open("clean_corpus.txt", "w", encoding="utf-8") as f:
    for doc in corpus:
        text = get_text(doc)
        if text:  # only write non-empty
            f.write(text + "\n\n")

# Compute stats
total_tokens = sum(len(get_text(doc).split()) for doc in corpus)
total_docs = len(corpus)
print(f"Total documents: {total_docs}")
print(f"Total tokens: {total_tokens}")



Total documents: 468
Total tokens: 114161
